Imports

In [1]:
!pip install sentence-transformers --quiet

In [2]:
from sentence_transformers import SentenceTransformer
import pandas as pd
import ast
import torch
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from string import punctuation

# Download necessary NLTK data files
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

Proverb Embeddings

In [3]:
proverbs_csv = "drive/MyDrive/annotated_proverbs.csv"
proverbs_df = pd.read_csv(proverbs_csv)

# Load a pretrained Sentence Transformer model
model = SentenceTransformer("all-MiniLM-L6-v2")

proverbs_df['topics'] = proverbs_df['topics'].apply(
    lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
proverbs_df['topics_string'] = proverbs_df['topics'].apply(lambda x: ', '.join(x))
data_for_training = (proverbs_df['text'] + proverbs_df['topics_string']).to_list()
embeddings = model.encode(data_for_training)
references = (proverbs_df['book'] + ' ' +
              proverbs_df['chapter'].astype(str) + ':' +
              proverbs_df['verse'].astype(str)).to_list()
proverbs = proverbs_df['text'].to_list()


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [4]:
def extract_keywords(sentence):
    # Define stop words and punctuation
    stop_words = set(stopwords.words('english'))

    # Tokenize the input sentence
    words = word_tokenize(sentence)

    # Filter out stop words and punctuation, keeping only alphabetic words
    keywords = [word for word in words if word.lower() not in stop_words and word not in punctuation and word.isalpha()]

    return ", ".join(keywords)

In [29]:
# Calculate embedding of query
prompt = input("Explain your quandry, and I will give you proverbs. Use precise language.\n\n")
prompt_keywords = extract_keywords(prompt)
prompt_embedding = model.encode(prompt_keywords)

# Calculate the embedding similarities
similarities = model.similarity(embeddings, prompt_embedding)

top_values, top_indices = torch.topk(similarities, 3, dim=0)

verse_tally=0
for j in range(3):
  ref = references[top_indices[j][0]]
  print(ref)
  print(proverbs[top_indices[j][0]])

  if '-' in ref:
    verse_tally += int(ref.split(':')[1].split('-')[1])-int(ref.split(':')[1].split('-')[0]) + 1
  else:
    verse_tally += 1
  if verse_tally > 2:
    break

Explain your quandry, and I will give you proverbs. Use precise language.

Good night


Proverbs 6:9
How long will you lie there, O sluggard? When will you arise from your sleep?

Proverbs 20:13
Love not sleep, lest you come to poverty; open your eyes, and you will have
plenty of bread.

Proverbs 26:14
As a door turns on its hinges, so does a sluggard on his bed.



Experimentation for Version 4